In [0]:
df_customer= spark.table("jpm.viraf_silver.customer_silver")
df_tran=spark.table("jpm.viraf_silver.transaction_silver")

In [0]:
%python
df_joined = df_customer.join(
    df_tran, 
    df_customer.cust_id == df_tran.cust_id, 
    "inner"
).select(
    df_customer.cust_id,
    'name',
    'age',
    'gender',
    'birthday',
    'zip',
    df_customer.city,
    'start_date',
    'end_date',
    'txn_id',
    'date',
    'year',
    'month',
    'day',
    'expense_type',
    'amt'
)

In [0]:
df_joined.createOrReplaceTempView("silver_joined")

In [0]:
%sql
create or replace table jpm.viraf_gold.all_records as 
select * from silver_joined

In [0]:
%sql
create or replace view jpm.viraf_gold.city_totals as 
select city, sum(amt) as total_sum from jpm.viraf_gold.all_records group by all

In [0]:
%sql
select * from jpm.viraf_gold.city_totals

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

In [0]:
%sql
create or replace view jpm.viraf_gold.cust_name_rank as
select name, city, RANK() OVER (partition by city ORDER BY SUM(amt) DESC) AS rank from jpm.viraf_gold.all_records group by name, city

In [0]:
%sql
select * from jpm.viraf_gold.cust_name_rank 